In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import math
import torch

In [2]:
# Define paths for input and output
VIDEO_DIR = "SportsData/"  # Directory containing MP4 videos
OUTPUT_DIR = "GraphData/"   # Directory to save graph data

FRAME_SKIP = 5

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [3]:
# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=2,  # Use the most accurate model
    smooth_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils

In [4]:
# Define the connections between body landmarks for graph creation
POSE_CONNECTIONS = [
    # Torso
    (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER),
    (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP),
    (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP),
    (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP),
    # Arms
    (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW),
    (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW),
    (mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
    (mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST),
    # Legs
    (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE),
    (mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE),
    (mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE),
    (mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE),
]

In [5]:
# Function to calculate angle between three points
def calculate_angle(a, b, c):
    """
    Calculate the angle between three points.
    Args:
        a: first point [x, y, z]
        b: middle point [x, y, z] (vertex of the angle)
        c: end point [x, y, z]
    Returns:
        angle in degrees
    """
    # Convert to numpy arrays
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    # Calculate vectors
    ba = a - b
    bc = c - b
    
    # Calculate dot product
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)  # Ensure value is within domain of arccos
    
    # Calculate angle in degrees
    angle = np.arccos(cosine_angle)
    angle = np.degrees(angle)
    
    return angle

In [6]:
# Define the key angles to calculate
def calculate_key_angles(landmarks):
    """
    Calculate key angles from landmarks.
    Args:
        landmarks: MediaPipe pose landmarks
    Returns:
        Dictionary of key angles
    """
    # Get positions as (x, y, z) tuples
    positions = {}
    for landmark in mp_pose.PoseLandmark:
        idx = landmark.value
        if idx < len(landmarks):
            positions[landmark] = [landmarks[idx].x, landmarks[idx].y, landmarks[idx].z]
    
    angles = {}
    
    # If any key landmarks are missing, return empty dict
    required_landmarks = [
        mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST,
        mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST,
        mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE,
        mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE
    ]
    
    if not all(landmark in positions for landmark in required_landmarks):
        return {}
    
    # Calculate elbow angles
    angles['left_elbow'] = calculate_angle(
        positions[mp_pose.PoseLandmark.LEFT_SHOULDER],
        positions[mp_pose.PoseLandmark.LEFT_ELBOW],
        positions[mp_pose.PoseLandmark.LEFT_WRIST]
    )
    
    angles['right_elbow'] = calculate_angle(
        positions[mp_pose.PoseLandmark.RIGHT_SHOULDER],
        positions[mp_pose.PoseLandmark.RIGHT_ELBOW],
        positions[mp_pose.PoseLandmark.RIGHT_WRIST]
    )
    
    # Calculate shoulder angles
    angles['left_shoulder'] = calculate_angle(
        positions[mp_pose.PoseLandmark.LEFT_HIP],
        positions[mp_pose.PoseLandmark.LEFT_SHOULDER],
        positions[mp_pose.PoseLandmark.LEFT_ELBOW]
    )
    
    angles['right_shoulder'] = calculate_angle(
        positions[mp_pose.PoseLandmark.RIGHT_HIP],
        positions[mp_pose.PoseLandmark.RIGHT_SHOULDER],
        positions[mp_pose.PoseLandmark.RIGHT_ELBOW]
    )
    
    # Calculate knee angles
    angles['left_knee'] = calculate_angle(
        positions[mp_pose.PoseLandmark.LEFT_HIP],
        positions[mp_pose.PoseLandmark.LEFT_KNEE],
        positions[mp_pose.PoseLandmark.LEFT_ANKLE]
    )
    
    angles['right_knee'] = calculate_angle(
        positions[mp_pose.PoseLandmark.RIGHT_HIP],
        positions[mp_pose.PoseLandmark.RIGHT_KNEE],
        positions[mp_pose.PoseLandmark.RIGHT_ANKLE]
    )
    
    # Calculate hip angles
    angles['left_hip'] = calculate_angle(
        positions[mp_pose.PoseLandmark.LEFT_SHOULDER],
        positions[mp_pose.PoseLandmark.LEFT_HIP],
        positions[mp_pose.PoseLandmark.LEFT_KNEE]
    )
    
    angles['right_hip'] = calculate_angle(
        positions[mp_pose.PoseLandmark.RIGHT_SHOULDER],
        positions[mp_pose.PoseLandmark.RIGHT_HIP],
        positions[mp_pose.PoseLandmark.RIGHT_KNEE]
    )
    
    return angles

In [7]:
# Function to create a graph from landmarks
def create_graph_from_landmarks(landmarks, angles, time_point, source_video, label):
    """
    Create a graph from MediaPipe pose landmarks.
    Args:
        landmarks: MediaPipe pose landmarks
        angles: Dictionary of calculated angles
        time_point: Time in the video
        source_video: Name of the source video
        label: 1 for hit, 0 for miss
    Returns:
        Graph data structure and node features
    """
    G = nx.Graph()
    
    # Add nodes with positional features
    for idx, landmark in enumerate(landmarks):
        G.add_node(idx, 
                  x=landmark.x, 
                  y=landmark.y, 
                  z=landmark.z, 
                  visibility=landmark.visibility)
    
    # Add edges according to pose connections
    for connection in POSE_CONNECTIONS:
        start_idx = connection[0].value
        end_idx = connection[1].value
        if start_idx < len(landmarks) and end_idx < len(landmarks):
            # Calculate Euclidean distance between nodes
            start_point = np.array([landmarks[start_idx].x, landmarks[start_idx].y, landmarks[start_idx].z])
            end_point = np.array([landmarks[end_idx].x, landmarks[end_idx].y, landmarks[end_idx].z])
            distance = np.linalg.norm(end_point - start_point)
            
            G.add_edge(start_idx, end_idx, weight=distance)
    
    # Prepare node features for GNN
    num_nodes = len(G.nodes())
    node_features = torch.zeros(num_nodes, 4)  # [x, y, z, visibility]
    
    for node in G.nodes():
        node_features[node, 0] = G.nodes[node]['x']
        node_features[node, 1] = G.nodes[node]['y']
        node_features[node, 2] = G.nodes[node]['z']
        node_features[node, 3] = G.nodes[node]['visibility']
    
    # Create edge index and edge attributes for PyTorch Geometric
    edge_index = []
    edge_attr = []
    
    for u, v, data in G.edges(data=True):
        edge_index.append([u, v])
        edge_index.append([v, u])  # Add in both directions for undirected graph
        
        edge_attr.append([data['weight']])
        edge_attr.append([data['weight']])
    
    if edge_index:  # Check if there are any edges
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(edge_attr, dtype=torch.float)
    else:
        edge_index = torch.zeros((2, 0), dtype=torch.long)
        edge_attr = torch.zeros((0, 1), dtype=torch.float)
    
    # Convert angles to tensor
    if angles:
        angle_keys = sorted(angles.keys())
        angle_tensor = torch.tensor([angles[k] for k in angle_keys], dtype=torch.float)
    else:
        angle_tensor = torch.zeros(8, dtype=torch.float)  # 8 angles we're calculating
    
    # Package everything together
    data = {
        'graph': G,
        'node_features': node_features,
        'edge_index': edge_index,
        'edge_attr': edge_attr,
        'angles': angle_tensor,
        'time': time_point,
        'source_video': source_video,
        'label': label
    }
    
    return data

In [8]:
# Modified process_video function (small change: return video name separately)
def process_video(video_path):
    """
    Process a video file to extract pose landmarks and create graph data.
    """
    video_name = os.path.basename(video_path)
    print(f"Processing {video_name} with frame skip = {FRAME_SKIP}...")

    # Determine label from video name
    if 'hit' in video_name.lower():
        label = 1  # Hit
    elif 'miss' in video_name.lower():
        label = 0  # Miss
    else:
        label = -1  # Unknown
        print(f"Warning: Could not determine label for {video_name}, defaulting to -1")

    # Open video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return [], video_name

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    all_graph_data = []
    frame_count = 0
    processed_count = 0

    with tqdm(total=total_frames // FRAME_SKIP + 1) as pbar:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            if frame_count % FRAME_SKIP == 0:
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = pose.process(rgb_frame)
                time_point = frame_count / fps

                if results.pose_landmarks:
                    landmarks = results.pose_landmarks.landmark
                    angles = calculate_key_angles(landmarks)
                    graph_data = create_graph_from_landmarks(
                        landmarks, angles, time_point, video_name, label
                    )
                    all_graph_data.append(graph_data)
                    processed_count += 1

                pbar.update(1)

            frame_count += 1

    cap.release()
    print(f"Processed {processed_count} frames out of {frame_count} total frames from {video_name}")

    return all_graph_data, video_name


In [9]:
# Modified main loop
def process_all_videos():
    """
    Process all videos in the input directory and save each video's graph data separately.
    """
    video_files = [f for f in os.listdir(VIDEO_DIR) if f.endswith('.mp4')]
    
    if not video_files:
        print(f"No mp4 files found in {VIDEO_DIR}")
        return
    
    print(f"Found {len(video_files)} videos to process")
    
    for video_file in video_files:
        video_path = os.path.join(VIDEO_DIR, video_file)
        
        # Process the video
        video_data, video_name = process_video(video_path)
        
        if not video_data:
            print(f"Warning: No graph data extracted from {video_name}")
            continue
        
        # Clean the video name to create a safe filename
        base_name = os.path.splitext(video_name)[0]
        output_file = os.path.join(OUTPUT_DIR, f"{base_name}_graph_data.pkl")
        
        # Save the graph data
        with open(output_file, 'wb') as f:
            pickle.dump(video_data, f)
        
        print(f"Saved graph data for {video_name} to {output_file}")

    print("Finished processing all videos.")


In [10]:
process_all_videos()

Found 25 videos to process
Processing Clip10Hit.mp4 with frame skip = 5...


 93%|█████████▎| 14/15 [00:00<00:00, 19.05it/s]


Processed 0 frames out of 70 total frames from Clip10Hit.mp4
Processing Clip11Hit.mp4 with frame skip = 5...


100%|██████████| 15/15 [00:02<00:00,  6.80it/s]


Processed 15 frames out of 73 total frames from Clip11Hit.mp4
Saved graph data for Clip11Hit.mp4 to GraphData/Clip11Hit_graph_data.pkl
Processing Clip12Hit.mp4 with frame skip = 5...


...

Processed 11 frames out of 59 total frames from Clip7Miss.mp4
Saved graph data for Clip7Miss.mp4 to GraphData/Clip7Miss_graph_data.pkl
Processing Clip8Hit.mp4 with frame skip = 5...


 94%|█████████▍| 15/16 [00:02<00:00,  6.90it/s]


Processed 14 frames out of 75 total frames from Clip8Hit.mp4
Saved graph data for Clip8Hit.mp4 to GraphData/Clip8Hit_graph_data.pkl
Processing Clip9Miss.mp4 with frame skip = 5...


100%|██████████| 9/9 [00:01<00:00,  7.28it/s]

Processed 9 frames out of 41 total frames from Clip9Miss.mp4
Saved graph data for Clip9Miss.mp4 to GraphData/Clip9Miss_graph_data.pkl
Finished processing all videos.
